In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sc
import math

plots_dir = 'S:\\lab_winter\\martin\\MED14_PRO-seq_final\\plots\\' # directory for output plots

In [50]:
#########################################
#### convert gene name to ENSEMBL ID ####
#########################################

dic = pd.read_csv('W:\lab_winter\private\Experiments\Matthias_MB\\2018\MB18_56 QuantSeq merge analysis\\data\\ENStoGeneName.txt', sep="\t")
dic.columns = ['geneID', 'gene_name']
dic = dic.set_index('gene_name')

In [88]:
############################
#### load in raw values ####
############################
## raw values generated with computeMatrix and plotHeatmap, but manually smoothed (running mean of +/- 3 bins) for the autoregTFs due to spiky signal (only 24 genes)

plot_data = pd.read_csv('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\KBM7_histone_ChIP-seq\\KBM7_H3K27ac_hg38_532_SuperEnhancers_expressed_genes_inTADS_maxTTseq_target_genes_renamed.bed', sep="\t", index_col=9).sort_values(by="enhancerRank")
plot_data = plot_data[plot_data.index != '.']

# DEseq2 dataframe
DEseq_TT_1h_df = pd.read_csv('S:\\lab_winter\\martin\\MJ-19-1\\20190418_JaegerWinter2019TTseqKBM7\\DifferentialExpression\\differential.expression.estimates_1h.txt', sep="\t")

SE_genes = pd.read_excel('S:\lab_winter\martin\Mediator_CRC\\SE_464_genes.xlsx')
##### same for 423 SE genes; unfortunately we lose one gene (not contained in the TT-seq annotation) #####
SE_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(SE_genes.geneID.tolist()), na=False)]

plot_data = plot_data.join(dic[dic.index.str.contains('|'.join(plot_data.index.tolist()), na=False)])
plot_data = plot_data[~plot_data.geneID.isna()]

plot_data = plot_data.join(DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(plot_data.geneID.tolist()), na=False)].set_index('gene_id').log2FoldChange, on="geneID").sort_values(by="enhancerRank")
plot_data.to_excel(plots_dir + "KBM_ChIP_SE-rank_vs_log2FC_raw_data.xlsx")

### manually clean up "plot_data.xlsx" to include genes where log2FC couldn't be automatically picked



In [272]:
DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('ENSG00000130640', na=False)].log2FoldChange

TR12806   -0.512863
Name: log2FoldChange, dtype: float64

In [103]:
plot_data[plot_data.log2FoldChange.isna()]

,CHROM,START,STOP,REGION_ID,enhancerRank,isSuper,KBM7_H3K27ac_signal,KBM7_input_signal,difference_H3K27ac_vs_input,RefSeq_id,TTseq_RPKM,geneID,log2FoldChange
gene_name,,,,,,,,,,,,,
MYB,chr6,135199523,135346208,36_KBM7_H3K27ac_hg38_peak_79585_lociStitched,9,1,130168.2690,17470.1835,112698.0,NM_005375,8.20751,ENSG00000118513,NaN
PTMA,chr2,231611447,231693648,22_KBM7_H3K27ac_hg38_peak_54201_lociStitched,27,1,81979.0573,6370.5775,75608.5,NM_002823,14.0869,ENSG00000187514,NaN
TOMM20,chr1,234968008,235025655,24_KBM7_H3K27ac_hg38_peak_8714_lociStitched,34,1,76970.2744,6220.1113,70750.2,NM_014765,0.988433,ENSG00000173726,NaN
MYH9,chr22,36327885,36475055,43_KBM7_H3K27ac_hg38_peak_59715_lociStitched,36,1,82106.1430,12789.0730,69317.1,NM_002473,0.832451,ENSG00000100345,NaN
ARHGAP22,chr10,48444595,48524943,21_KBM7_H3K27ac_hg38_peak_10572_lociStitched,41,1,70240.2216,7560.7468,62679.5,NM_001256026,0.567738,ENSG00000128805,NaN
TLE3,chr15,70424431,70536965,27_KBM7_H3K27ac_hg38_peak_30244_lociStitched,44,1,83781.5630,21674.0484,62107.5,NM_001282982,0.483398,ENSG00000140332,NaN
CXXC5,chr5,139654598,139713626,18_KBM7_H3K27ac_hg38_peak_73549_lociStitched,47,1,65892.9564,5808.3552,60084.6,NM_001317211,0.895539,ENSG00000171604,NaN
PTPRC,chr1,198597543,198685375,18_KBM7_H3K27ac_hg38_peak_6954_lociStitched,50,1,69606.8600,11822.1872,57784.7,NM_001267798,2.05023,ENSG00000081237,NaN
PTPRC,chr1,198597543,198685375,18_KBM7_H3K27ac_hg38_peak_6954_lociStitched,50,1,69606.8600,11822.1872,57784.7,NM_001267798,2.05023,ENSG00000262418,NaN


In [18]:
plot_data.groupby("gene_name", sort=False).first()

,CHROM,START,STOP,enhancerRank,isSuper,KBM7_H3K27ac_signal,KBM7_input_signal,difference_H3K27ac_vs_input,RefSeq_id,TTseq_RPKM
gene_name,,,,,,,,,,
MYC,chr8,129533853,129691821,1,1,358492.5792,36601.1856,321891.0,NM_002467,4.08605
SATB1,chr3,17891450,18166677,2,1,275584.7951,33440.0805,242145.0,NM_002971,2.92036
SLC39A10,chr2,195312430,195583082,3,1,262451.2444,30664.8716,231786.0,NM_020342,0.507407
ETV6,chr12,11656261,11896930,4,1,227023.0677,26232.9210,200790.0,NM_001987,2.64913
LIPC,chr15,58437002,58612842,5,1,176754.3680,19465.4880,157289.0,NM_000236,0.118544
RHEX,chr1,205980465,206116040,6,1,151355.9300,14777.6750,136578.0,NM_001007544,1.89707
VSIG4,chrX,65969260,66055572,7,1,139488.8232,10754.4752,128734.0,NM_007268,0.369956
GSE1,chr16,85515731,85617915,8,1,129017.5184,9584.8592,119433.0,NM_014615,0.556301
MYB,chr6,135199523,135346208,9,1,130168.2690,17470.1835,112698.0,NM_005375,8.20751


In [17]:
plot_data.groupby("gene_name", sort=False).mean()

,START,STOP,enhancerRank,isSuper,KBM7_H3K27ac_signal,KBM7_input_signal,difference_H3K27ac_vs_input
gene_name,,,,,,,
MYC,1.295339e+08,1.296918e+08,1.0,1.0,358492.57920,36601.185600,321891.000000
SATB1,1.803947e+07,1.821645e+07,8.5,1.0,190936.49755,21884.859450,169051.800000
SLC39A10,1.954724e+08,1.956192e+08,245.5,1.0,141941.94455,16614.072550,125327.700000
ETV6,1.165626e+07,1.189693e+07,4.0,1.0,227023.06770,26232.921000,200790.000000
LIPC,5.843700e+07,5.861284e+07,5.0,1.0,176754.36800,19465.488000,157289.000000
RHEX,2.059805e+08,2.061160e+08,6.0,1.0,151355.93000,14777.675000,136578.000000
VSIG4,6.596926e+07,6.605557e+07,7.0,1.0,139488.82320,10754.475200,128734.000000
GSE1,8.539636e+07,8.545812e+07,227.0,1.0,61738.42300,5339.678733,56398.866667
MYB,1.351995e+08,1.353462e+08,9.0,1.0,130168.26900,17470.183500,112698.000000
